In [1]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-19.0")
user

UserTO(id=caec7d23-c8b4-4777-9cd5-e5969ecd82ca, username=Johann-19.0)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=85aea154-8379-4c7f-82b6-a387bda5a9c3, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=a8e47b99-102b-41e9-bc99-6226953dc6d4, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=d123ce9c-d58c-4751-9ffe-267c0e2a1140, name=ING RON, unit=Currency(value=RON))
AccountTO(id=bf9836ab-ff52-4295-bd06-15c165ab4f2f, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=98b9aac1-6e34-435a-ae80-2a7d062b5577, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=e6799116-e708-4777-8f14-d7720a162b68, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=895c57cc-45f6-4f0c-b8a5-266874569f83, name=Other, unit=Currency(value=RON))
AccountTO(id=480aaf60-8a7a-4fd6-aadb-dcd62b384de1, name=BT RON, unit=Currency(value=RON))
AccountTO(id=e2553fc7-1c46-42c5-827e-8fe3a8283e3b, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=ecf21adf-ac0e-4805-a2b3-78679a70929b, name=Expenses)
FundTO(id=1d6a00b4-1f01-4d46-8545-041a847cab16, name=Work Income)
FundTO(id=27cdf000-b605-46ea-a023-defdd6e3126d, name=Savings)
FundTO(id=890bbdd4-6cab-4492-bec9-bb907b480e3f, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=e01bd587-0bf7-4405-825f-475ff0f80944, status=COMPLETED, reason=null)